In [1]:
!ls

Test-Ganomaly.ipynb  data  ganomaly


In [2]:
!git clone https://github.com/samet-akcay/ganomaly.git

fatal: destination path 'ganomaly' already exists and is not an empty directory.


In [3]:
!conda

/bin/sh: 1: conda: not found


In [4]:
!pip install -r ganomaly/requirements.txt

     |████████████████████████████████| 157 kB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 430 kB 10.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.3 MB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 5.2 MB/s eta 0:00:01
     |████████████████████████████████| 278 kB 7.5 MB/s eta 0:00:01
     |████████████████████████████████| 90 kB 8.3 MB/s  eta 0:00:01
     |████████████████████████████████| 13.1 MB 7.1 MB/s eta 0:00:01
     |████████████████████████████████| 17.3 MB 8.0 MB/s eta 0:00:01
     |████████████████████████████████| 112 kB 6.6 MB/s eta 0:00:01
     |████████████████████████████████| 158 kB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 1.7 MB/s eta 0:00:011
     |████████████████████████████████| 226 kB 8.5 MB/s eta 0:00:01
     |████████████████████████████████| 510 kB 7.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 

      Successfully uninstalled cryptography-2.1.4
  Attempting uninstall: idna
    Found existing installation: idna 2.6
    Uninstalling idna-2.6:
      Successfully uninstalled idna-2.6
  Attempting uninstall: kiwisolver
    Found existing installation: kiwisolver 1.3.1
    Uninstalling kiwisolver-1.3.1:
      Successfully uninstalled kiwisolver-1.3.1
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Uninstalling python-dateutil-2.8.1:
      Successfully uninstalled python-dateutil-2.8.1
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 2.4.7
    Uninstalling pyparsing-2.4.7:
      Successfully uninstalled pyparsing-2.4.7
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.4
    Uninstalling numpy-1.19.4:
      Successfully uninstalled numpy-1.19.4
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.3.3
    Uninstalling matplotlib-3.3.3:
      Successfully un

In [5]:
!pip install mkl-fft

     |████████████████████████████████| 232 kB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 6.1 MB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 425 kB 8.9 MB/s eta 0:00:01
     |████████████████████████████████| 737.3 MB 5.3 kB/s eta 0:00:011
     |████████████████████████████████| 11.2 MB 8.5 MB/s eta 0:00:01
     |████████████████████████████████| 396 kB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 3.6 MB 5.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 5.2 MB/s eta 0:00:01
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

icc-rt 2020.0.133 requires intel-openmp==2020.*, but you'll have intel-openmp 2021.1.2 which is incompatible.
You should consider upgrading via the '/usr/bin

In [6]:
import os
import torch
import numpy as np
import torchvision.datasets as datasets
from torchvision.datasets import MNIST
from torchvision.datasets import CIFAR10
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

In [7]:
# params
OPT_isize = 32
OPT_abnormal_class = "plane"
OPT_manualseed = 55
OPT_batchsize = 64
OPT_workers = 8

splits = ['train', 'test']
drop_last_batch = {'train': True, 'test': False}
shuffle = {'train': True, 'test': False}

In [8]:
transform = transforms.Compose(
    [
        transforms.Resize(OPT_isize),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

classes = {
    'plane': 0, 'car': 1, 'bird': 2, 'cat': 3, 'deer': 4,
    'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9
}

In [9]:
dataset = {}
dataset['train'] = CIFAR10(root='./data', train=True, download=True, transform=transform)
dataset['test'] = CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
##
def get_cifar_anomaly_dataset(trn_img, trn_lbl, tst_img, tst_lbl, abn_cls_idx=0, manualseed=-1):
    """[summary]
    Arguments:
        trn_img {np.array} -- Training images
        trn_lbl {np.array} -- Training labels
        tst_img {np.array} -- Test     images
        tst_lbl {np.array} -- Test     labels
    Keyword Arguments:
        abn_cls_idx {int} -- Anomalous class index (default: {0})
    Returns:
        [np.array] -- New training-test images and labels.
    """
    # Convert train-test labels into numpy array.
    trn_lbl = np.array(trn_lbl)
    tst_lbl = np.array(tst_lbl)

    # --
    # Find idx, img, lbl for abnormal and normal on org dataset.
    nrm_trn_idx = np.where(trn_lbl != abn_cls_idx)[0]
    abn_trn_idx = np.where(trn_lbl == abn_cls_idx)[0]
    nrm_trn_img = trn_img[nrm_trn_idx]    # Normal training images
    abn_trn_img = trn_img[abn_trn_idx]    # Abnormal training images
    nrm_trn_lbl = trn_lbl[nrm_trn_idx]    # Normal training labels
    abn_trn_lbl = trn_lbl[abn_trn_idx]    # Abnormal training labels.

    nrm_tst_idx = np.where(tst_lbl != abn_cls_idx)[0]
    abn_tst_idx = np.where(tst_lbl == abn_cls_idx)[0]
    nrm_tst_img = tst_img[nrm_tst_idx]    # Normal training images
    abn_tst_img = tst_img[abn_tst_idx]    # Abnormal training images.
    nrm_tst_lbl = tst_lbl[nrm_tst_idx]    # Normal training labels
    abn_tst_lbl = tst_lbl[abn_tst_idx]    # Abnormal training labels.

    # --
    # Assign labels to normal (0) and abnormals (1)
    nrm_trn_lbl[:] = 0
    nrm_tst_lbl[:] = 0
    abn_trn_lbl[:] = 1
    abn_tst_lbl[:] = 1

    # --
    if manualseed != -1:
        # Random seed.
        # Concatenate the original train and test sets.
        nrm_img = np.concatenate((nrm_trn_img, nrm_tst_img), axis=0)
        nrm_lbl = np.concatenate((nrm_trn_lbl, nrm_tst_lbl), axis=0)
        abn_img = np.concatenate((abn_trn_img, abn_tst_img), axis=0)
        abn_lbl = np.concatenate((abn_trn_lbl, abn_tst_lbl), axis=0)

        # Split the normal data into the new train and tests.
        idx = np.arange(len(nrm_lbl))
        np.random.seed(manualseed)
        np.random.shuffle(idx)

        nrm_trn_len = int(len(idx) * 0.80)
        nrm_trn_idx = idx[:nrm_trn_len]
        nrm_tst_idx = idx[nrm_trn_len:]

        nrm_trn_img = nrm_img[nrm_trn_idx]
        nrm_trn_lbl = nrm_lbl[nrm_trn_idx]
        nrm_tst_img = nrm_img[nrm_tst_idx]
        nrm_tst_lbl = nrm_lbl[nrm_tst_idx]

    # Create new anomaly dataset based on the following data structure:
    # - anomaly dataset
    #   . -> train
    #        . -> normal
    #   . -> test
    #        . -> normal
    #        . -> abnormal
    new_trn_img = np.copy(nrm_trn_img)
    new_trn_lbl = np.copy(nrm_trn_lbl)
    new_tst_img = np.concatenate((nrm_tst_img, abn_trn_img, abn_tst_img), axis=0)
    new_tst_lbl = np.concatenate((nrm_tst_lbl, abn_trn_lbl, abn_tst_lbl), axis=0)

    return new_trn_img, new_trn_lbl, new_tst_img, new_tst_lbl

In [11]:
dataset['train'].data, dataset['train'].targets, \
dataset['test'].data, dataset['test'].targets = get_cifar_anomaly_dataset(
    trn_img=dataset['train'].data,
    trn_lbl=dataset['train'].targets,
    tst_img=dataset['test'].data,
    tst_lbl=dataset['test'].targets,
    abn_cls_idx=classes[OPT_abnormal_class],
    manualseed=OPT_manualseed
)

In [12]:
dataloader = {x: torch.utils.data.DataLoader(dataset=dataset[x],
                                             batch_size=OPT_batchsize,
                                             shuffle=shuffle[x],
                                             num_workers=int(OPT_workers),
                                             drop_last=drop_last_batch[x],
                                             worker_init_fn=(None if OPT_manualseed == -1
                                             else lambda x: np.random.seed(OPT_manualseed)))
              for x in splits}

In [13]:
type(dataloader)

dict

In [14]:
dataloader.keys()

dict_keys(['train', 'test'])

In [15]:
dir(dataloader['train'])

['_DataLoader__initialized',
 '_DataLoader__multiprocessing_context',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_auto_collation',
 '_index_sampler',
 'batch_sampler',
 'batch_size',
 'collate_fn',
 'dataset',
 'dataset_kind',
 'drop_last',
 'multiprocessing_context',
 'num_workers',
 'pin_memory',
 'sampler',
 'timeout',
 'worker_init_fn']

In [16]:
print(type(dataloader))
type(dataloader['train'])

<class 'dict'>


torch.utils.data.dataloader.DataLoader

In [17]:
# iterate through and print few of the data types stored within the iterator to examine it
# Create a similar iterator for the OASIS-3 no-ad scans directory